<a href="https://colab.research.google.com/github/VeroFrias93/TOPICO-DE-LA-INDUSTRIA-1/blob/main/Practica_4_VMFP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** VERONICA MARISELA FRIAS PERALTA

**e-mail:** veronica.frias6148@alumnos.udg.mx

##MODULES

In [1]:
import panel as pn
import panel.widgets as pnw
import param
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from math import pi

import holoviews as hv
pn.extension(sizing_mode = 'stretch_width')

from shapely.geometry import Point, LineString

##CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

##FUNCTIONS

In [3]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def bm_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)


    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    # return dataframe
    return BM_2d_df

In [4]:
def crwv1(pasos: int = 200, velocidad: float = 4, posicion_inicial: list = [0, 0], exp_crw: float = 0.7) -> pd.DataFrame:
  """
Caminata Aleatoria Correlacionada (CRW) en 2 dimensiones.

  Argumentos:
      pasos (int): Número de pasos a simular.
      velocidad (float): Velocidad de la CRW.
      posicion_inicial (list): Posición inicial (x, y).
      exp_crw (float): Exponente de la distribución de Cauchy.

  Retorna:
      df_crw (pd.DataFrame): Trayectoria de la CRW con las columnas "x_pos" e "y_pos".
  """

  # Move velocidad definition outside the loop
  velocidad_actual = np.array([velocidad, 0])
  posicion_inicial = np.array(posicion_inicial)
  df_crw = pd.DataFrame(columns=["x_pos", "y_pos"])

  bm_2d = np.random.uniform(low=-np.pi, high=np.pi, size=pasos)

  for i in range(pasos):
      angulo_actual = bm_2d[i]
      cambio_angulo = angulo_actual
      velocidad_actual = velocidad_actual * np.array([np.cos(cambio_angulo), np.sin(cambio_angulo)])

      nueva_posicion = posicion_inicial + velocidad_actual * i
      df_crw = df_crw.append({"x_pos": nueva_posicion[0], "y_pos": nueva_posicion[1]}, ignore_index=True)

  return df_crw


  # Vuelo de Levy
def lvfv1(pasos=200, velocidad=4, s_pos=[0,0], cauchy_lv=0.4, exp_lv = 1):
    """
    Argumentos:
        pasos: Número de pasos.
        velocidad: Velocidad.
        posicion_inicial: Posición inicial.
        cauchy_lv: Coeficiente Cauchy para el Vuelo de Levy.
        exp_lv: Exponente del Vuelo de Levy.
    Retorna:
        df_vuelo_levy: Dataframe con la trayectoria del Vuelo de Levy.
    """

    velocidad = Vec2d(velocidad, 0)
    std_mstps = 3
    df_vuelo_levy = pd.DataFrame(columns=["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": 0, "y_pos": 0}])
    df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
    i = 1
    while i < pasos:
        angulo_giro = wrapcauchy.rvs(c=cauchy_lv, loc=0)
        velocidad = velocidad.rotated(angulo_giro)
        n_pasos = abs(math.floor(levy_stable.rvs(alpha=exp_lv, beta=1, loc=std_mstps)))
        if (i + n_pasos > pasos):
            n_pasos = pasos - 1

        c_paso = 0
        while c_paso < n_pasos:
            temp_df = pd.DataFrame([{"x_pos": df_vuelo_levy.x_pos[i-1] + velocidad.x,
                                     "y_pos": df_vuelo_levy.y_pos[i-1] + velocidad.y}])
            df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
            c_paso += 1
            i += 1

    return df_vuelo_levy




In [5]:
n_steps=pnw.IntInput(name='Number of steps',value=200,step=2,start=0,end=2000)
speed = pnw.IntInput(name='Speed', value=8,step=1,start=0,end=10)
x_pos = pnw.FloatInput(name='Starting pos_x',value=0,step=1,start=0,end=1000)
y_pos = pnw.FloatInput(name='starting pos_y',value=0,step=1,start=0,end=1000)
crw_exp =pnw.FloatInput(name='Cauchy coefficient',value=0.6, step=0.2, start=0.5, end=1)
lv_exp = pnw.FloatInput(name='Levy exponent',value=0.6,step=0.6,start=0.5,end=1)


##Trajectories

In [7]:
# BM
@pn.depends(n_steps, speed, x_pos, y_pos)
def plot_traj(n_steps, speed, x_pos, y_pos):
    fig_traj_bm = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps,speed,[x_pos, y_pos])

    fig_traj_bm.add_trace(
        go.Scatter3d(x=bm_df.x_pos,
                     y=bm_df.y_pos,
                     z=bm_df.index,
                     marker=dict(size=2),
                     line=dict(color='#pink', width=2),
                     mode='lines',
                     name= f'BM={n_steps}',

                     showlegend=True))

    fig_traj_bm.update_traces(groupnorm='fraction',
               selector=dict(type='scatter'))
    return fig_traj_bm


    # CRW
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def plot_traj_crw(n_steps, speed, x_pos, y_pos, crw_exp):
    fig_traj_crw = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    fig_traj_crw.add_trace(
        go.Scatter3d(x=crw_df.x_pos,
                     y=crw_df.y_pos,
                     z=crw_df.index,
                     marker=dict(size=2),
                     line=dict(color='#pink', width=2),
                     mode='lines',
                     name= f'CRW= {crw_exp}',
                     showlegend=True))

    fig_traj_crw.update_traces(groupnorm='fraction',
                            selector=dict(type='scatter'))
    return fig_traj_crw